# Class IV - An introduction to GasModels.jl

*Los Alamos National Laboratory Grid Science Winter School, 2019*

Welcome! This tutorial will introduce you to the basics of the [GasModels.jl](https://github.com/lanl-ansi/GasModels.jl) package. If you haven't yet, work through [Class I - An introduction  to Julia](Class%20I%20-%20An%20introduction%20to%20Julia.ipynb) and [Class II - An introduction  to JuMP](Class%20II%20-%20An%20introduction%20to%20JuMP.ipynb) first.

As in Class II, run the following magic sauce to check we're good to go.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
println("Excellent! Everything is good to go!")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Excellent! Everything is good to go!


### Background 

This [presentation](https://github.com/lanl-ansi/tutorial-grid-science-2019/blob/master/assets/infrastructure_optimization_in_julia.pdf) provides motivation and context for this notebook.

Some additional informaiton is also available at,
* [GasModels Documentation](https://lanl-ansi.github.io/GasModels.jl/stable/)


### Working with the Network Model

A 40-node gas network models is provided with this tutorial.  The MATLAB-like input data files can be viewed [here](../edit/data/gaslib40.m).

Similar to PowerModels, the `parse_file` function from GasModels is used to load a text files into the GasModels data model, 

In [2]:
using GasModels

data = GasModels.parse_file("data/gaslib40.m")

[warn | InfrastructureModels]: Matlab parser skipping the following line:
  end
[warn | GasModels]: no case version found in .m file.  The file seems to be missing "mgc.version = ..."


Dict{String,Any} with 21 entries:
  "gas_specific_gravity"         => 0.6
  "temperature"                  => 273.15
  "source_type"                  => ".m"
  "name"                         => "gaslib40"
  "source_version"               => "0.0.0+"
  "multinetwork"                 => false
  "gas_molar_mass"               => 0.0185674
  "compressor"                   => Dict{String,Any}("41"=>Dict{String,Any}("t_…
  "consumer"                     => Dict{String,Any}("24"=>Dict{String,Any}("ql…
  "standard_density"             => 1.0
  "baseQ"                        => 604.167
  "compressibility_factor"       => 0.8
  "specific_heat_capacity_ratio" => 1.4
  "producer"                     => Dict{String,Any}("1"=>Dict{String,Any}("qgm…
  "per_unit"                     => true
  "junction"                     => Dict{String,Any}("32"=>Dict{String,Any}("ju…
  "pipe"                         => Dict{String,Any}("32"=>Dict{String,Any}("le…
  "baseP"                        => 8101325
  "sound

In this case the file parser generated warning messages.  

The second warning block message indicates that the data file did not have any version information. Still the parser will load this data.

Now, lets look at some of the network data.  The data for the first pipe and compressor can be viewed as follows,

In [3]:
data["connection"]["1"]

ArgumentError: ArgumentError: invalid index: 1 of type String

In [4]:
data["connection"]["100000"]

ArgumentError: ArgumentError: invalid index: 100000 of type String

Here, connection can either be of type "pipe" in which case it has a length, diameter, and friction_factor associated with it or can be a compressor with a different set of fields as shown above.

The first junction in the network can be viewed as follows,

In [5]:
data["junction"]["1"]

Dict{String,Any} with 7 entries:
  "junction_type" => 0
  "status"        => 1
  "junction_i"    => 1
  "pmax"          => 1.0
  "p_nominal"     => 3.10133e6
  "pmin"          => 0.382817
  "index"         => 1

That's great, but looking at components one-by-one can get boring fast.

All InfrastructureModels packages provide a `print_summary` function that prints a table-like summary of the network data to the terminal.

In [6]:
GasModels.print_summary(data)

Metadata
  R: 8.314
  baseP: 8101325
  baseQ: 604.167
  compressibility_factor: 0.800
  connection: [(0)]
  gas_molar_mass: 0.019
  gas_specific_gravity: 0.600
  multinetwork: false
  name: gaslib40
  per_unit: true
  sound_speed: 312.805
  source_type: .m
  source_version: 0.0.0+
  specific_heat_capacity_ratio: 1.400
  standard_density: 1.000
  temperature: 273.150

Table Counts
  junction: 46
  producer: 3
  consumer: 29
  compressor: 12
  pipe: 39


Table: junction
          junction_i,  pmin,  pmax,  p_nominal
       0:          0, 0.013, 1.000, 101325.000
       1:          1, 0.383, 1.000, 3.101325e6
       2:          2, 0.383, 1.000, 3.101325e6
       3:          3, 0.013, 1.000, 101325.000
       4:          4, 0.013, 1.000, 101325.000
       5:          5, 0.383, 1.000, 3.101325e6
       6:          6, 0.013, 1.000, 101325.000
       7:          7, 0.013, 1.000, 101325.000
       8:          8, 0.013, 1.000, 101325.000
       9:          9, 0.013, 1.000, 101325.000
      10: 

### Solving the Gas Flow Problem (GF)

Before we can solve a gas flow problem, we need a solver.  The gas flow problem as defined in GasModels.jl is a Mixed-Integer Nonlinear Program (MINLP). Hence, we require an MINLP solver; in this case we will use [Pavito](https://github.com/JuliaOpt/Pavito.jl). This MINLP solver requires two other solvers (1) a mip-solver and (2) a continuous NLP solver. We shall use GLPK and Ipopt as the mip solver and NLP solvers, respectively. 

In [ ]:
using JuMP
using Cbc
using Ipopt
using Juniper

cbc_solver = JuMP.with_optimizer(Cbc.Optimizer, logLevel=0)
ipopt_solver = JuMP.with_optimizer(Ipopt.Optimizer, tol=1e-6, print_level=0)

juniper_solver = JuMP.with_optimizer(Juniper.Optimizer, mip_solver=cbc_solver, nl_solver=ipopt_solver)

result = run_gf(data, MINLPGasModel, juniper_solver)

nl_solver   : OptimizerFactory(Ipopt.Optimizer, (), Base.Iterators.Pairs{Symbol,Real,Tuple{Symbol,Symbol},NamedTuple{(:tol, :print_level),Tuple{Float64,Int64}}}(:tol=>1.0e-6,:print_level=>0))
mip_solver  : OptimizerFactory(Cbc.Optimizer, (), Base.Iterators.Pairs(:logLevel=>0))
log_levels  : Symbol[:Options, :Table, :Info]

#Variables: 199
#IntBinVar: 102
#Constraints: 565
#Linear Constraints: 409
#Quadratic Constraints: 0
#NonLinear Constraints: 156
Obj Sense: Max

Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 0.05903792381286621
Relaxation Obj: 0.0

       MIPobj              NLPobj       Time 
      29.9912              0.0115        0.1 
       2.3716              0.0043        0.3 
       2.2784              0.0057        0.5 
       2.3312              0.0043        0.8 
       2.0951              0.003         1.2 
       2.1906              0.0005        1.7 
       2.1161              0.0013        2.2 
       2.141               0.0113        2.9 
       2.3434    

The result object contains a variety of useful information about the optimization problem solved, including the objective value and wall clock runtime.  Detailed documentation of the result dictionary is available [here](https://lanl-ansi.github.io/GasModels.jl/stable/result-data/), however most interesting point is the `solution` data.

In [13]:
result["solution"]

UndefVarError: UndefVarError: result not defined

Not every useful, lets try `print_summary`,

In [14]:
GasModels.print_summary(result["solution"])

UndefVarError: UndefVarError: result not defined